### Install Required Libraries

In [13]:
# Install required libraries if not already installed
!pip install requests pandas tqdm

### Load Wallets from CSV

In [9]:
# Import all required modules
import pandas as pd
import requests
from tqdm import tqdm
import time

wallet_df = pd.read_csv("Wallet id - Sheet1.csv")
wallet_list = wallet_df['wallet_id'].dropna().unique().tolist()
wallet_list[:5]  # preview first 5

### Set Covalent API Details

In [1]:
COVALENT_API_KEY = "cqt_rQHKjfpqGbfVCQFjm8Txgbc68CgH"
CHAIN_ID = 1  # Ethereum Mainnet

### Define Function to Fetch Compound V2 Transactions

In [13]:
import requests
from tqdm import tqdm

def get_wallet_transactions(wallet_address):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet_address}/transactions_v2/"
    params = {
        "key": COVALENT_API_KEY
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"[ERROR] HTTP {response.status_code} for wallet: {wallet_address}")
            return None
        return response.json()
    except Exception as e:
        print(f"[ERROR] Wallet: {wallet_address}, {e}")
        return None

### Collect Data for All Wallets

In [15]:
wallet_data = {}

for wallet in tqdm(wallet_list[:10]):  # test on first 10
    data = get_wallet_transactions(wallet)
    if data and data.get('data', {}).get('items'):
        wallet_data[wallet] = data['data']['items']

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.37s/it]


### Create Features From Transactions

In [17]:
wallet_features = []

for wallet, txns in wallet_data.items():
    total_txns = len(txns)
    total_gas = sum(txn.get("gas_spent", 0) for txn in txns if txn.get("gas_spent"))
    successful_txns = sum(1 for txn in txns if txn.get("successful") == True)

    wallet_features.append({
        "wallet_id": wallet,
        "total_txns": total_txns,
        "successful_txns": successful_txns,
        "success_ratio": successful_txns / total_txns if total_txns else 0,
        "total_gas": total_gas
    })

features_df = pd.DataFrame(wallet_features)
features_df.head()

,wallet_id,total_txns,successful_txns,success_ratio,total_gas
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,100,99,0.99,235826553
1,0x06b51c6882b27cb05e712185531c1f74996dd988,5,5,1.00,308732
2,0x0795732aacc448030ef374374eaae57d2965c16c,4,4,1.00,290793
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,80,80,1.00,132280277
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,5,5,1.00,308720


### Normalize Features & Score Wallets

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
score_features = features_df[["total_txns", "successful_txns", "success_ratio", "total_gas"]]
features_df_scaled = pd.DataFrame(scaler.fit_transform(score_features), columns=score_features.columns)

features_df["score"] = (features_df_scaled * [0.3, 0.3, 0.3, 0.1]).sum(axis=1) * 1000
features_df["score"] = features_df["score"].astype(int)


### Save to CSV

In [21]:
final_df = features_df[["wallet_id", "score"]]
final_df.to_csv("wallet_scores.csv", index=False)
final_df.head()

,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,849
1,0x06b51c6882b27cb05e712185531c1f74996dd988,306
2,0x0795732aacc448030ef374374eaae57d2965c16c,300
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,833
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,306


### Explanation